# Data Load

In [2]:
import pandas as pd
import urllib.request, json
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import nltk, gensim
from nltk import word_tokenize

C:\Users\Moon\Anaconda3\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [18]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [5]:
os.chdir('C:\\Users\\Moon\\Google 드라이브\\경배끄10\\인턴\\csv파일\\리뷰들\\SUV리뷰')
data = open('all_suv_review2(15~17).json').read()
review = pd.DataFrame(json.loads(data))
print(review.shape)

(306, 30)


In [7]:
os.chdir('C:\\Users\\Moon\\Google 드라이브\\경배끄10\\인턴\\csv파일\\리뷰들\\SUV리뷰')
data = open('all_suv_review(15~17).json').read()
review2 = pd.DataFrame(json.loads(data))
print(review2.shape)

(1281, 30)


In [14]:
SUV = pd.concat([review, review2])
print(SUV.shape)
SUV.head(1)

(1587, 30)


,author,averageRating,carDetailsDto,comfortRatingDto,commentsCount,created,favoriteFeatures,id,interiorRatingDto,legacyId,...,suggestedImprovements,technologyRatingDto,text,thumbsUpDownCounter,title,updated,userRating,valueRatingDto,version,year_dummy
1,"{'authorName': 'Brian125', 'cognitoId': 'us-ea...",4,None,"{'comfortRating': 4.0, 'frontSeats': 'OKAY', '...",0.0,1.499772e+12,None,1197426175753150464,"{'interiorRating': 4.0, 'cargoStorage': 'OKAY'...",1197426175753150464,...,None,"{'technologyRating': 4.0, 'entertainment': 'OK...",Babysitting my friend's new Edge till I pick h...,"{'thumbsDown': 0, 'thumbsUp': 1}",Not a bad Cross over,1.500380e+12,4.0,{},0.0,2017.0


# Word2Vec으로 리뷰분류하기
* Step1 : Word2Vec을 활용하여 리뷰 각 단어들을 Clustering한다.
* Step2 : 각 단어들의 유사성을 비교하여 각 단어의 Scoring을 하고, **유사도(가중치)** 매트릭스를 만든다.
* Step3 : TDM 을 만들어 각 리뷰별 해당 단어가 빈도수를 체크하는 Matrix를 만든다
* Step4 : 유사도 Matrix와 TDM을 **내적**하여 각 리뷰별 단어의 중요도를 평가하여 해당 리뷰를 특정 영역으로 **Clustering**한다.

In [15]:
# text 문장만 추출하기
SUV = SUV['text'].dropna()

In [19]:
# 문장 토크나이징
sentence = [ nltk.word_tokenize(x) for x in SUV.dropna()]

In [26]:
# Word2Vec모델 학습

model = gensim.models.Word2Vec(sentence, min_count=1, size=10, window=2, workers=4, sg=1)
model

In [27]:
model_index = model.wv.index2word


model_vector = pd.DataFrame(model[model_index], index=model_index)
print(model_vector.shape)
model_vector.head(5)

(13913, 10)


,0,1,2,3,4,5,6,7,8,9
.,-0.226704,0.230382,-0.413961,-1.858366,-0.873917,0.046866,0.801484,0.185909,-0.250639,0.056180
the,-0.731911,1.430056,-0.012164,-0.928676,-0.621849,-0.542365,0.300486,-0.186563,-0.314534,0.999775
",",-0.955894,0.842828,-0.034094,-1.479844,-0.842591,0.378638,0.027919,-0.097858,-0.233479,0.555398
and,-1.038178,1.103890,0.187990,-1.259900,-0.914072,0.230184,-0.211842,-0.144520,-0.147221,0.660897
I,0.456331,1.310812,0.347615,-2.273553,-0.688694,-0.253066,-0.288988,-1.042796,-0.829935,0.667914


## 가중치 매트릭스

In [28]:
from scipy.spatial import distance
from tqdm import tqdm

In [ ]:
words = model_vector.index
func = distance.euclidean
sim_matrix = []
for word in tqdm(words):
    similarites = []
    word_vec = model_vector.loc[word].values.tolist()
    for word2 in words:
        word2_vec = model_vector.loc[word2].values.tolist()
        similarites.append(round(func(word_vec, word2_vec),5))

    sim_matrix.append(similarites)


  4%|███▏                                                                       | 586/13913 [30:20<12:13:09,  3.30s/it]

In [171]:
from sklearn.preprocessing import normalize

In [172]:
normal_sim = pd.DataFrame((normalize(sim_matrix)),index=model_vector.index, columns=model_vector.index)
normal_sim.head(4)

,the,.,",",and,a,to,I,is,it,of,...,cleaning,Feb.,settled,area,prevalent,pick,Given,number,satisfied,paint
the,0.000000,0.028004,0.027345,0.025487,0.033772,0.031786,0.029761,0.030264,0.026388,0.035174,...,0.021118,0.021078,0.023097,0.020677,0.020459,0.021022,0.021264,0.021371,0.024876,0.022169
.,0.027273,0.000000,0.031736,0.029256,0.028022,0.032405,0.029093,0.028179,0.028188,0.035407,...,0.020197,0.021838,0.022592,0.021998,0.021236,0.022581,0.021724,0.022524,0.020802,0.022475
",",0.025290,0.030137,0.000000,0.022710,0.031101,0.032456,0.030667,0.026478,0.025663,0.034209,...,0.021754,0.023189,0.023325,0.021634,0.021842,0.020842,0.022055,0.022862,0.023873,0.021626
and,0.025745,0.030345,0.024805,0.000000,0.033417,0.031359,0.028511,0.030276,0.027963,0.033776,...,0.020710,0.022424,0.022481,0.020581,0.022139,0.020246,0.021206,0.021377,0.024627,0.021866


In [173]:
normal_sim.to_csv('word2vec_model3.csv')

## 데이터 다시불러들이기

In [274]:
## Matrix 불러오기
os.chdir('C:\\Users\\kb910\\Google 드라이브\\경배끄10\\인턴\\csv파일\\complain')
normal_sim = pd.read_csv('word2vec_model2.csv')


In [275]:
test = normal_sim['Unnamed: 0'].values
normal_sim = normal_sim.drop(['Unnamed: 0'], axis=1)
normal_sim.index = test

# TDM 만들기 / 목표 단어 가중치 매트릭스만들기

In [276]:
test = [ str(x) for x in Maxcruz2.values.tolist() ] 

voca = normal_sim.columns

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(vocabulary=voca)
context = test
X = vectorizer.fit_transform(context)
TDM_matrix = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())

for column in TDM_matrix.columns:
    TDM_matrix.loc[TDM_matrix[column] != 0, column] = 1
    
TDM_matrix.shape

(155, 3196)

In [277]:
columns = ['cargo','price','interior','technology']
ext_sim = normal_sim[columns]

test = pd.DataFrame(np.dot(TDM_matrix, ext_sim), columns=columns)
print(test.shape)
test.head(5)

(155, 4)


,cargo,price,interior,technology
0,1.172884,1.321155,1.272589,1.199085
1,4.178007,4.743160,4.603252,4.443945
2,3.688302,4.042762,4.013352,3.865160
3,1.377639,1.495099,1.482581,1.469132
4,0.293908,0.319489,0.306339,0.272668


In [174]:
len(test2[test2['cargo']==1])

'C:\\Users\\kb910\\Google 드라이브\\경배끄10\\인턴\\csv파일\\complain'